In [1]:
import pandas as pd 
import requests
from jsonschema import validate
import json
import fastparquet

In [54]:
pd.set_option('display.width', None)


In [31]:
countriesDf = pd.DataFrame(pd.read_parquet('../dataRawBeforeBlob/countries/euCountriesRaw.parquet'))
countriesDf

,Code,Title,ParentDimension,Dimension,ParentCode,ParentTitle
0,AUT,Austria,REGION,COUNTRY,EUR,Europe
1,BEL,Belgium,REGION,COUNTRY,EUR,Europe
2,BGR,Bulgaria,REGION,COUNTRY,EUR,Europe
3,CYP,Cyprus,REGION,COUNTRY,EUR,Europe
4,CZE,Czechia,REGION,COUNTRY,EUR,Europe
5,DEU,Germany,REGION,COUNTRY,EUR,Europe
6,DNK,Denmark,REGION,COUNTRY,EUR,Europe
7,ESP,Spain,REGION,COUNTRY,EUR,Europe
8,EST,Estonia,REGION,COUNTRY,EUR,Europe
9,FIN,Finland,REGION,COUNTRY,EUR,Europe


In [9]:
tobaccoIndicatorsDf = pd.DataFrame(pd.read_parquet('../dataRawBeforeBlob/indicators/tobaccoIndicators.parquet',engine='fastparquet'))
tobaccoIndicatorsDf

,IndicatorCode,IndicatorName,Language,Category
0,R_afford_gdp,Affordability - percentage of GDP per capita r...,EN,Tobacco control: Raise taxes on tobacco
1,TOB_R_PRICE,Retail price for tobacco and nicotine products,EN,Tobacco control: Raise taxes on tobacco
2,E1_nat_tv_radio,Ban on advertising: National TV and radio,EN,Tobacco control: Enforce bans on tobacco adver...
3,E6a_pt_of_sale,Ban on advertising at point of sale,EN,Tobacco control: Enforce bans on tobacco adver...
4,E5_billboards,Ban on advertising: Billboards and outdoor adv...,EN,Tobacco control: Enforce bans on tobacco adver...
5,E7_internet,Ban on advertising: Internet,EN,Tobacco control: Enforce bans on tobacco adver...
6,P6_restaurants,National smoking ban in RESTAURANTS,EN,Tobacco control: Protect from tobacco smoke
7,P8_pub_transp,National smoking ban in PUBLIC TRANSPORT,EN,Tobacco control: Protect from tobacco smoke
8,P5_indoor_off,National smoking ban in INDOOR OFFICES,EN,Tobacco control: Protect from tobacco smoke


In [2]:
baseURL= 'https://ghoapi.azureedge.net/api/'

In [28]:
def getData(baseurl,indicatorCode:str):
    indicatorCode = indicatorCode.strip()
    response = requests.get(baseurl+f"{indicatorCode}")
    return response.json()

In [40]:
def saveToParquet(df,filename):
    df.to_parquet(f'../dataRawBeforeBlob/tobaccoData/{filename}.parquet',engine='fastparquet')

First indicator

In [17]:
firstIndicator = tobaccoIndicatorsDf.iloc[0]
firstIndicator

IndicatorCode                                         R_afford_gdp
IndicatorName    Affordability - percentage of GDP per capita r...
Language                                                        EN
Category                   Tobacco control: Raise taxes on tobacco
Name: 0, dtype: object

In [39]:
firstIndicatorData = getData(baseURL,firstIndicator['IndicatorCode'])
firstIndicatorData = firstIndicatorData['value']
firstIndicatorDataDataFrame = pd.DataFrame(firstIndicatorData)
firstIndicatorDataDataFrame =  firstIndicatorDataDataFrame.loc[firstIndicatorDataDataFrame['SpatialDim'].isin(countriesDf['Code'])]
firstIndicatorDataDataFrame = firstIndicatorDataDataFrame[['Id','IndicatorCode','SpatialDim','TimeDim','Value']]
firstIndicatorDataDataFrame

,Id,IndicatorCode,SpatialDim,TimeDim,Value
4,1898831,R_afford_gdp,GRC,2014,2.47
7,2785705,R_afford_gdp,BEL,2014,1.61
40,298771,R_afford_gdp,NLD,2014,1.58
45,4405899,R_afford_gdp,LUX,2018,0.53
66,10142682,R_afford_gdp,DEU,2018,1.58
...,...,...,...,...,...
1132,4747273,R_afford_gdp,IRL,2016,1.91
1134,8925205,R_afford_gdp,GRC,2020,2.98
1136,8324747,R_afford_gdp,CZE,2020,2.02
1141,9083506,R_afford_gdp,DEU,2020,1.71


In [42]:
saveToParquet(firstIndicatorDataDataFrame,'affordability')

In [43]:
secondIndicator = tobaccoIndicatorsDf.iloc[1]
secondIndicator 

IndicatorCode                                       TOB_R_PRICE
IndicatorName    Retail price for tobacco and nicotine products
Language                                                     EN
Category                Tobacco control: Raise taxes on tobacco
Name: 1, dtype: object

In [46]:
secondIndicatorData = getData(baseURL,secondIndicator['IndicatorCode'])
secondIndicatorData = secondIndicatorData['value']
secondIndicatorData

[{'Id': 8859,
  'IndicatorCode': 'TOB_R_PRICE',
  'SpatialDimType': 'COUNTRY',
  'SpatialDim': 'KWT',
  'ParentLocationCode': 'EMR',
  'TimeDimType': 'YEAR',
  'ParentLocation': 'Eastern Mediterranean',
  'Dim1Type': 'TOBACCO_NICOTINE_PRODUCT',
  'TimeDim': 2016,
  'Dim1': 'TOBACCO_NICOTINE_PRODUCT_CIGARETTE_MOSTSOLD',
  'Dim2Type': 'PRICE',
  'Dim2': 'PRICE_CURRENCY',
  'Dim3Type': None,
  'Dim3': None,
  'DataSourceDimType': None,
  'DataSourceDim': None,
  'Value': 'KWD',
  'NumericValue': None,
  'Low': None,
  'High': None,
  'Comments': None,
  'Date': '2023-12-15T15:32:24+01:00',
  'TimeDimensionValue': '2016',
  'TimeDimensionBegin': '2016-01-01T00:00:00+01:00',
  'TimeDimensionEnd': '2016-12-31T00:00:00+01:00'},
 {'Id': 14624,
  'IndicatorCode': 'TOB_R_PRICE',
  'SpatialDimType': 'COUNTRY',
  'SpatialDim': 'SSD',
  'ParentLocationCode': 'AFR',
  'TimeDimType': 'YEAR',
  'ParentLocation': 'Africa',
  'Dim1Type': 'TOBACCO_NICOTINE_PRODUCT',
  'TimeDim': 2022,
  'Dim1': 'TOBACCO_

In [60]:
secondIndicatorDataDataFrame = pd.DataFrame(secondIndicatorData)
secondIndicatorDataDataFrame = secondIndicatorDataDataFrame.loc[secondIndicatorDataDataFrame['SpatialDim'].isin(countriesDf['Code'])]
secondIndicatorDataDataFrame = secondIndicatorDataDataFrame[secondIndicatorDataDataFrame['Dim1'] == 'TOBACCO_NICOTINE_PRODUCT_CIGARETTE_MOSTSOLD']
secondIndicatorDataDataFrame = secondIndicatorDataDataFrame[secondIndicatorDataDataFrame['Dim2'] == 'PRICE_PRICE_IN_USD']
secondIndicatorDataDataFrame = secondIndicatorDataDataFrame[['Id','IndicatorCode','SpatialDim','TimeDim','Dim1','Dim2','Value']]
secondIndicatorDataDataFrame

,Id,IndicatorCode,SpatialDim,TimeDim,Dim1,Dim2,Value
40,122023,TOB_R_PRICE,BEL,2008,TOBACCO_NICOTINE_PRODUCT_CIGARETTE_MOSTSOLD,PRICE_PRICE_IN_USD,5.79
51,161442,TOB_R_PRICE,LVA,2018,TOBACCO_NICOTINE_PRODUCT_CIGARETTE_MOSTSOLD,PRICE_PRICE_IN_USD,4.11
204,621897,TOB_R_PRICE,IRL,2018,TOBACCO_NICOTINE_PRODUCT_CIGARETTE_MOSTSOLD,PRICE_PRICE_IN_USD,14.32
221,659841,TOB_R_PRICE,SVN,2012,TOBACCO_NICOTINE_PRODUCT_CIGARETTE_MOSTSOLD,PRICE_PRICE_IN_USD,3.58
271,793859,TOB_R_PRICE,LTU,2008,TOBACCO_NICOTINE_PRODUCT_CIGARETTE_MOSTSOLD,PRICE_PRICE_IN_USD,2.04
...,...,...,...,...,...,...,...
12467,9259986,TOB_R_PRICE,FRA,2010,TOBACCO_NICOTINE_PRODUCT_CIGARETTE_MOSTSOLD,PRICE_PRICE_IN_USD,7.3
12506,9446326,TOB_R_PRICE,BGR,2014,TOBACCO_NICOTINE_PRODUCT_CIGARETTE_MOSTSOLD,PRICE_PRICE_IN_USD,3.01
12521,9499630,TOB_R_PRICE,ESP,2014,TOBACCO_NICOTINE_PRODUCT_CIGARETTE_MOSTSOLD,PRICE_PRICE_IN_USD,6.42
12542,9592819,TOB_R_PRICE,DEU,2022,TOBACCO_NICOTINE_PRODUCT_CIGARETTE_MOSTSOLD,PRICE_PRICE_IN_USD,7.75


In [61]:
saveToParquet(secondIndicatorDataDataFrame,'mostSoldCigarettePrice')

In [62]:
thirdIndicator = tobaccoIndicatorsDf.iloc[2]
thirdIndicator

IndicatorCode                                         E1_nat_tv_radio
IndicatorName               Ban on advertising: National TV and radio
Language                                                           EN
Category         Tobacco control: Enforce bans on tobacco advertising
Name: 2, dtype: object

In [64]:
thirdIndicatorData = getData(baseURL,thirdIndicator['IndicatorCode'])
thirdIndicatorData = thirdIndicatorData['value']
thirdIndicatorData

[{'Id': 6289190,
  'IndicatorCode': 'E1_nat_tv_radio',
  'SpatialDimType': 'COUNTRY',
  'SpatialDim': 'UGA',
  'ParentLocationCode': 'AFR',
  'TimeDimType': 'YEAR',
  'ParentLocation': 'Africa',
  'Dim1Type': None,
  'TimeDim': 2018,
  'Dim1': None,
  'Dim2Type': None,
  'Dim2': None,
  'Dim3Type': None,
  'Dim3': None,
  'DataSourceDimType': None,
  'DataSourceDim': None,
  'Value': 'Yes',
  'NumericValue': None,
  'Low': None,
  'High': None,
  'Comments': None,
  'Date': '2023-10-31T14:40:22+01:00',
  'TimeDimensionValue': '2018',
  'TimeDimensionBegin': '2018-01-01T00:00:00+01:00',
  'TimeDimensionEnd': '2018-12-31T00:00:00+01:00'},
 {'Id': 254437,
  'IndicatorCode': 'E1_nat_tv_radio',
  'SpatialDimType': 'COUNTRY',
  'SpatialDim': 'LTU',
  'ParentLocationCode': 'EUR',
  'TimeDimType': 'YEAR',
  'ParentLocation': 'Europe',
  'Dim1Type': None,
  'TimeDim': 2022,
  'Dim1': None,
  'Dim2Type': None,
  'Dim2': None,
  'Dim3Type': None,
  'Dim3': None,
  'DataSourceDimType': None,
  'Da

In [67]:
thirdIndicatorDataDataFrame = pd.DataFrame(thirdIndicatorData)
thirdIndicatorDataDataFrame = thirdIndicatorDataDataFrame[thirdIndicatorDataDataFrame['SpatialDim'].isin(countriesDf['Code'])]
thirdIndicatorDataDataFrame = thirdIndicatorDataDataFrame[['Id','IndicatorCode','SpatialDim','TimeDim','Value']]
thirdIndicatorDataDataFrame

,Id,IndicatorCode,SpatialDim,TimeDim,Value
1,254437,E1_nat_tv_radio,LTU,2022,Yes
19,7706327,E1_nat_tv_radio,POL,2008,Yes
20,5051170,E1_nat_tv_radio,LTU,2010,Yes
24,615532,E1_nat_tv_radio,LUX,2010,Yes
25,4096217,E1_nat_tv_radio,NLD,2007,Yes
...,...,...,...,...,...
1727,9751492,E1_nat_tv_radio,GRC,2010,Yes
1733,9906574,E1_nat_tv_radio,POL,2018,Yes
1736,9994551,E1_nat_tv_radio,BEL,2016,Yes
1753,10192422,E1_nat_tv_radio,CZE,2022,Yes


In [ ]:
saveToParquet(thirdIndicatorDataDataFrame,'TobaccoAdversting')